In [1]:
import geopandas as gpd
import momepy as mm
from tqdm import tqdm
from momepy import limit_range
import numpy as np
import pandas as pd
from inequality.theil import Theil
import libpysal
import scipy as sp
import mapclassify
import mapclassify.classifiers as classifiers

In [2]:
blg = gpd.read_parquet('../../nairobi/buildings.pq')
streets = gpd.read_parquet('../../nairobi/edges.pq')
tess = gpd.read_parquet('../../nairobi/tessellation.pq')
blocks = gpd.read_parquet('../../nairobi/blocks.pq')

## Measure

In [3]:
blg['sdbAre'] = mm.Area(blg).series
blg['sdbPer'] = mm.Perimeter(blg).series
blg['ssbCCo'] = mm.CircularCompactness(blg, 'sdbAre').series
blg['ssbCor'] = mm.Corners(blg).series
blg['ssbSqu'] = mm.Squareness(blg).series
blg['ssbERI'] = mm.EquivalentRectangularIndex(blg, 'sdbAre', 'sdbPer').series
blg['ssbElo'] = mm.Elongation(blg).series

 46%|████▌     | 230978/507532 [00:32<00:39, 7017.49it/s]/opt/conda/lib/python3.8/site-packages/momepy/shape.py:773: RuntimeWarning: invalid value encountered in arccos
  angle = np.arccos(cosine_angle)
 46%|████▌     | 231102/507532 [00:52<01:01, 4523.21it/s]/opt/conda/lib/python3.8/site-packages/momepy/shape.py:862: RuntimeWarning: invalid value encountered in arccos
  angle = np.degrees(np.arccos(cosine_angle))
100%|██████████| 507532/507532 [01:59<00:00, 4237.94it/s]


In [4]:
cencon = mm.CentroidCorners(blg)
blg['ssbCCM'] = cencon.mean
blg['ssbCCD'] = cencon.std

 46%|████▌     | 231144/507532 [01:07<01:22, 3366.12it/s]/opt/conda/lib/python3.8/site-packages/momepy/shape.py:1080: RuntimeWarning: invalid value encountered in arccos
  angle = np.arccos(cosine_angle)
100%|██████████| 507532/507532 [02:32<00:00, 3325.20it/s]


In [5]:
blg['stbOri'] = mm.Orientation(blg).series
tess['stcOri'] = mm.Orientation(tess).series
blg['stbCeA'] = mm.CellAlignment(blg, tess, 'stbOri', 'stcOri', 'uID', 'uID').series

100%|██████████| 506435/506435 [06:32<00:00, 1290.98it/s]


In [6]:
tess['sdcLAL'] = mm.LongestAxisLength(tess).series
tess['sdcAre'] = mm.Area(tess).series
tess['sscCCo'] = mm.CircularCompactness(tess, 'sdcAre').series
tess['sscERI'] = mm.EquivalentRectangularIndex(tess, 'sdcAre').series
tess['sicCAR'] = mm.AreaRatio(tess, blg, 'sdcAre', 'sdbAre', 'uID').series

In [7]:
queen_1 = libpysal.weights.contiguity.Queen.from_dataframe(tess, ids="uID", silence_warnings=True)
 
blg["mtbAli"] = mm.Alignment(blg, queen_1, "uID", "stbOri").series
blg["mtbNDi"] = mm.NeighborDistance(blg, queen_1, "uID").series
tess["mtcWNe"] = mm.Neighbors(tess, queen_1, "uID", weighted=True).series
tess["mdcAre"] = mm.CoveredArea(tess, queen_1, "uID").series

100%|██████████| 506435/506435 [01:25<00:00, 5941.44it/s]


In [9]:
blocks["ldkAre"] = mm.Area(blocks).series
blocks["ldkPer"] = mm.Perimeter(blocks).series
blocks["lskCCo"] = mm.CircularCompactness(blocks, "ldkAre").series
blocks["lskERI"] = mm.EquivalentRectangularIndex(blocks, "ldkAre", "ldkPer").series
blocks["lskCWA"] = mm.CompactnessWeightedAxis(blocks, "ldkAre", "ldkPer").series
blocks["ltkOri"] = mm.Orientation(blocks).series
 
blo_q1 = libpysal.weights.contiguity.Queen.from_dataframe(blocks, ids="bID", silence_warnings=True)
 
blocks["ltkWNB"] = mm.Neighbors(blocks, blo_q1, "bID", weighted=True).series
blocks["likWBB"] = mm.Count(blocks, blg, "bID", "bID", weighted=True).series

100%|██████████| 12191/12191 [00:00<00:00, 96794.87it/s]


In [11]:
tess.drop(columns='geometry').to_parquet('../../nairobi/tess_data.parquet')
blg.drop(columns='geometry').to_parquet('../../nairobi/blg_data.parquet')
blocks.drop(columns='geometry').to_parquet('../../nairobi/blocks_data.parquet')

<ipython-input-11-5b763fe89c32>:1: UserWarning: this is an initial implementation of Parquet/Feather file support and associated metadata.  This is tracking version 0.1.0 of the metadata specification at https://github.com/geopandas/geo-arrow-spec

This metadata specification does not yet make stability promises.  We do not yet recommend using this in a production setting unless you are able to rewrite your Parquet/Feather files.

To further ignore this warning, you can do: 
import warnings; warnings.filterwarnings('ignore', message='.*initial implementation of Parquet.*')
  tess.drop(columns='geometry').to_parquet('../../nairobi/tess_data.parquet')
<ipython-input-11-5b763fe89c32>:2: UserWarning: this is an initial implementation of Parquet/Feather file support and associated metadata.  This is tracking version 0.1.0 of the metadata specification at https://github.com/geopandas/geo-arrow-spec

This metadata specification does not yet make stability promises.  We do not yet recommend us

In [12]:
queen3 = mm.sw_high(k=3, weights=queen_1)
queen1 = queen_1
blg_queen = blg_q1

blg['ltbIBD'] = mm.MeanInterbuildingDistance(blg, queen1, 'uID', queen3).series
blg['ltcBuA'] = mm.BuildingAdjacency(blg, queen3, 'uID', blg_queen).series

  0%|          | 324/507532 [00:00<05:12, 1625.52it/s]

Computing mean interbuilding distances...


Calculating adjacency: 100%|██████████| 507532/507532 [00:03<00:00, 156483.19it/s]


In [13]:
tess = tess.merge(blg[['uID']], on='uID', how='left')

tess['ltcWRB'] = mm.BlocksCount(tess, 'bID', queen3, 'uID').series

100%|██████████| 506435/506435 [06:22<00:00, 1325.50it/s]


In [14]:
tess.drop(columns='geometry').to_parquet('../../nairobi/tess_data.parquet')
blg.drop(columns='geometry').to_parquet('../../nairobi/blg_data.parquet')
 
fo = libpysal.io.open('../../nairobi/queen1.gal', 'w')
fo.write(queen1)
fo.close()
 
fo = libpysal.io.open('../../nairobi/queen3.gal', 'w')
fo.write(queen3)
fo.close()
 
fo = libpysal.io.open('../../nairobi/blg_queen.gal', 'w')
fo.write(blg_queen)
fo.close()

<ipython-input-14-bd7e78ee6eca>:1: UserWarning: this is an initial implementation of Parquet/Feather file support and associated metadata.  This is tracking version 0.1.0 of the metadata specification at https://github.com/geopandas/geo-arrow-spec

This metadata specification does not yet make stability promises.  We do not yet recommend using this in a production setting unless you are able to rewrite your Parquet/Feather files.

To further ignore this warning, you can do: 
import warnings; warnings.filterwarnings('ignore', message='.*initial implementation of Parquet.*')
  tess.drop(columns='geometry').to_parquet('../../nairobi/tess_data.parquet')
<ipython-input-14-bd7e78ee6eca>:2: UserWarning: this is an initial implementation of Parquet/Feather file support and associated metadata.  This is tracking version 0.1.0 of the metadata specification at https://github.com/geopandas/geo-arrow-spec

This metadata specification does not yet make stability promises.  We do not yet recommend us

In [15]:
streets["sdsLen"] = mm.Perimeter(streets).series
tess["stcSAl"] = mm.StreetAlignment(tess, streets, "stcOri", "nID").series
blg["stbSAl"] = mm.StreetAlignment(blg, streets, "stbOri", "nID").series

profile = mm.StreetProfile(streets, blg, distance=3)
streets["sdsSPW"] = profile.w
streets["sdsSPO"] = profile.o
streets["sdsSWD"] = profile.wd
 
streets["sssLin"] = mm.Linearity(streets).series
streets["sdsAre"] = mm.Reached(streets, tess, "nID", "nID", mode="sum", values="sdcAre").series
streets["sisBpM"] = mm.Count(streets, blg, "nID", "nID", weighted=True).series

/opt/conda/lib/python3.8/site-packages/numpy/lib/nanfunctions.py:1664: RuntimeWarning: Degrees of freedom <= 0 for slice.
  var = nanvar(a, axis=axis, dtype=dtype, out=out, ddof=ddof,
/opt/conda/lib/python3.8/site-packages/momepy/dimension.py:626: RuntimeWarning: invalid value encountered in long_scalars
  openness.append(np.isnan(s).sum() / (f).sum())
100%|██████████| 115518/115518 [05:14<00:00, 366.90it/s]


In [16]:
tess.drop(columns='geometry').to_parquet('../../nairobi/tess_data.parquet')
blg.drop(columns='geometry').to_parquet('../../nairobi/blg_data.parquet')
streets.drop(columns='geometry').to_parquet('../../nairobi/streets_data.parquet')

<ipython-input-16-6c060e936c79>:1: UserWarning: this is an initial implementation of Parquet/Feather file support and associated metadata.  This is tracking version 0.1.0 of the metadata specification at https://github.com/geopandas/geo-arrow-spec

This metadata specification does not yet make stability promises.  We do not yet recommend using this in a production setting unless you are able to rewrite your Parquet/Feather files.

To further ignore this warning, you can do: 
import warnings; warnings.filterwarnings('ignore', message='.*initial implementation of Parquet.*')
  tess.drop(columns='geometry').to_parquet('../../nairobi/tess_data.parquet')
<ipython-input-16-6c060e936c79>:2: UserWarning: this is an initial implementation of Parquet/Feather file support and associated metadata.  This is tracking version 0.1.0 of the metadata specification at https://github.com/geopandas/geo-arrow-spec

This metadata specification does not yet make stability promises.  We do not yet recommend us

In [17]:
str_q1 = libpysal.weights.contiguity.Queen.from_dataframe(streets)
 
streets["misRea"] = mm.Reached(
    streets, tess, "nID", "nID", spatial_weights=str_q1, mode="count"
).series
streets["mdsAre"] = mm.Reached(streets, tess, "nID", "nID", spatial_weights=str_q1,
                               mode="sum").series

100%|██████████| 115518/115518 [14:26<00:00, 133.26it/s]


In [18]:
graph = mm.gdf_to_nx(streets)
 
print("node degree")
graph = mm.node_degree(graph)
 
print("subgraph")
graph = mm.subgraph(
    graph,
    radius=5,
    meshedness=True,
    cds_length=False,
    mode="sum",
    degree="degree",
    length="mm_len",
    mean_node_degree=False,
    proportion={0: True, 3: True, 4: True},
    cyclomatic=False,
    edge_node_ratio=False,
    gamma=False,
    local_closeness=True,
    closeness_weight="mm_len",
)
print("cds length")
graph = mm.cds_length(graph, radius=3, name="ldsCDL")
 
print("clustering")
graph = mm.clustering(graph, name="xcnSCl")
 
print("mean_node_dist")
graph = mm.mean_node_dist(graph, name="mtdMDi")
 
nodes, edges, sw = mm.nx_to_gdf(graph, spatial_weights=True)
 
print("saving")
nodes.to_parquet('../../nairobi/g_nodes.pq')
edges.to_parquet('../../nairobi/g_edges.pq')
 
fo = libpysal.io.open('../../nairobi/nodes.gal', "w")
fo.write(sw)
fo.close()
 
edges_w3 = mm.sw_high(k=3, gdf=edges)
edges["ldsMSL"] = mm.SegmentsLength(edges, spatial_weights=edges_w3, mean=True).series
 
edges["ldsRea"] = mm.Reached(edges, tess, "nID", "nID", spatial_weights=edges_w3).series
edges["ldsRea"] = mm.Reached(
    edges, tess, "nID", "nID", spatial_weights=edges_w3, mode="sum", values="sdcAre"
).series
 
nodes_w5 = mm.sw_high(k=5, weights=sw)
nodes["lddNDe"] = mm.NodeDensity(nodes, edges, nodes_w5).series
nodes["linWID"] = mm.NodeDensity(
    nodes, edges, nodes_w5, weighted=True, node_degree="degree"
).series
 
blg["nodeID"] = mm.get_node_id(blg, nodes, edges, "nodeID", "nID")
tess = tess.merge(blg[["uID", "nodeID"]], on="uID", how="left")
 
nodes_w3 = mm.sw_high(k=3, weights=sw)
 
nodes["lddRea"] = mm.Reached(nodes, tess, "nodeID", "nodeID", nodes_w3).series
nodes["lddARe"] = mm.Reached(
    nodes, tess, "nodeID", "nodeID", nodes_w3, mode="sum", values="sdcAre"
).series
 
nodes["sddAre"] = mm.Reached(
    nodes, tess, "nodeID", "nodeID", mode="sum", values="sdcAre"
).series
nodes["midRea"] = mm.Reached(nodes, tess, "nodeID", "nodeID", spatial_weights=sw).series
nodes["midAre"] = mm.Reached(
    nodes, tess, "nodeID", "nodeID", spatial_weights=sw, mode="sum", values="sdcAre"
).series
 
nodes.rename(
    columns={
        "degree": "mtdDeg",
        "meshedness": "lcdMes",
        "local_closeness": "lcnClo",
        "proportion_3": "linP3W",
        "proportion_4": "linP4W",
        "proportion_0": "linPDE",
    }, inplace=True
)

node degree
subgraph


100%|██████████| 53367/53367 [01:12<00:00, 739.29it/s] 


cds length


100%|██████████| 53367/53367 [00:27<00:00, 1938.84it/s]


clustering
mean_node_dist


100%|██████████| 53367/53367 [00:00<00:00, 68858.41it/s]


saving


<ipython-input-18-5e698dbfc976>:35: UserWarning: this is an initial implementation of Parquet/Feather file support and associated metadata.  This is tracking version 0.1.0 of the metadata specification at https://github.com/geopandas/geo-arrow-spec

This metadata specification does not yet make stability promises.  We do not yet recommend using this in a production setting unless you are able to rewrite your Parquet/Feather files.

To further ignore this warning, you can do: 
import warnings; warnings.filterwarnings('ignore', message='.*initial implementation of Parquet.*')
  nodes.to_parquet('../../nairobi/g_nodes.pq')
<ipython-input-18-5e698dbfc976>:36: UserWarning: this is an initial implementation of Parquet/Feather file support and associated metadata.  This is tracking version 0.1.0 of the metadata specification at https://github.com/geopandas/geo-arrow-spec

This metadata specification does not yet make stability promises.  We do not yet recommend using this in a production sett

In [19]:
tess.drop(columns='geometry').to_parquet('../../nairobi/tess_data.parquet')
blg.drop(columns='geometry').to_parquet('../../nairobi/blg_data.parquet')
nodes.drop(columns='geometry').to_parquet('../../nairobi/nodes_data.parquet')
edges.drop(columns='geometry').to_parquet('../../nairobi/edges_data.parquet')

<ipython-input-19-f1c2eb66ab77>:1: UserWarning: this is an initial implementation of Parquet/Feather file support and associated metadata.  This is tracking version 0.1.0 of the metadata specification at https://github.com/geopandas/geo-arrow-spec

This metadata specification does not yet make stability promises.  We do not yet recommend using this in a production setting unless you are able to rewrite your Parquet/Feather files.

To further ignore this warning, you can do: 
import warnings; warnings.filterwarnings('ignore', message='.*initial implementation of Parquet.*')
  tess.drop(columns='geometry').to_parquet('../../nairobi/tess_data.parquet')
<ipython-input-19-f1c2eb66ab77>:2: UserWarning: this is an initial implementation of Parquet/Feather file support and associated metadata.  This is tracking version 0.1.0 of the metadata specification at https://github.com/geopandas/geo-arrow-spec

This metadata specification does not yet make stability promises.  We do not yet recommend us

In [20]:
merged = tess.merge(blg.drop(columns=['nID', 'bID', 'nodeID', 'geometry']), on='uID')
merged = merged.merge(blocks.drop(columns='geometry'), on='bID', how='left')
merged = merged.merge(edges.drop(columns='geometry'), on='nID', how='left')
merged = merged.merge(nodes.drop(columns='geometry'), on='nodeID', how='left')

In [21]:
primary = merged.drop(columns=['nID', 'bID', 'nodeID', 'mm_len', 'cdsbool', 'node_start', 'node_end', 'geometry'])
primary.to_parquet('../../nairobi/primary.parquet')

<ipython-input-21-52050a5a525f>:2: UserWarning: this is an initial implementation of Parquet/Feather file support and associated metadata.  This is tracking version 0.1.0 of the metadata specification at https://github.com/geopandas/geo-arrow-spec

This metadata specification does not yet make stability promises.  We do not yet recommend using this in a production setting unless you are able to rewrite your Parquet/Feather files.

To further ignore this warning, you can do: 
import warnings; warnings.filterwarnings('ignore', message='.*initial implementation of Parquet.*')
  primary.to_parquet('../../nairobi/primary.parquet')


## Contextual

In [2]:
def theil(y):
    y = np.array(y)
    n = len(y)
    plus = y + np.finfo('float').tiny * (y == 0)  # can't have 0 values
    yt = plus.sum(axis=0)
    s = plus / (yt * 1.0)
    lns = np.log(n * s)
    slns = s * lns
    t = sum(slns)
    return t

In [3]:
def _simpson_di(data):

    def p(n, N):
        if n == 0:
            return 0
        return float(n) / N

    N = sum(data.values())

    return sum(p(n, N) ** 2 for n in data.values() if n != 0)

In [4]:
primary = pd.read_parquet('../../nairobi/primary.parquet')

In [5]:
geom = gpd.read_parquet('../../nairobi/tessellation.pq', columns=["geometry"])

In [6]:
queen = libpysal.weights.Queen.from_dataframe(geom)

/opt/conda/lib/python3.8/site-packages/libpysal/weights/weights.py:172: UserWarning: The weights matrix is not fully connected: 
 There are 139 disconnected components.
 There are 71 islands with ids: 3010, 11189, 13679, 13825, 14590, 15478, 17987, 19134, 19899, 20722, 20817, 24942, 29564, 32396, 36253, 119727, 164897, 168291, 174530, 174799, 177215, 187304, 188359, 190129, 191485, 334820, 344385, 350734, 364525, 407230, 409817, 410981, 411555, 420979, 430138, 431771, 434375, 439154, 441308, 443173, 443610, 445775, 451986, 454222, 457594, 463478, 475539, 483899, 484682, 488039, 495762, 498974, 500077, 500235, 500353, 500526, 500961, 501305, 501944, 502540, 502814, 502859, 504701, 505683, 505951, 506046, 506228, 506375, 506377, 506413, 506414.
  warnings.warn(message)


In [7]:
wk = sum(map(lambda x: queen.sparse ** x, range(2, 11)))

In [8]:
spatial_weights = libpysal.weights.WSP(wk).to_W()

In [9]:
gdf = primary

In [10]:
gdf = gdf.drop(columns="highway")

In [11]:
gdf = gdf.fillna(0)  # normally does not happen, but to be sure
chars = gdf.columns

In [12]:
skewness = pd.DataFrame(index=chars)
for c in chars:
    skewness.loc[c, 'skewness'] = sp.stats.skew(gdf[c])
headtail = list(skewness.loc[skewness.skewness >= 1].index)
to_invert = skewness.loc[skewness.skewness <= -1].index

for inv in to_invert:
    gdf[inv + '_r'] = gdf[inv].max() - gdf[inv]
inverted = [x for x in gdf.columns if '_r' in x]
headtail = headtail + inverted
natural = [x for x in chars if x not in headtail]

In [13]:
bins = {}
for c in headtail:
    bins[c] = mapclassify.HeadTailBreaks(gdf[c]).bins
for c in natural:
    bins[c] = mapclassify.gadf(gdf[c], method='NaturalBreaks')[1].bins

In [14]:
means = {}
ranges = {}
theils = {}
simpsons = {}

In [15]:
for ch in gdf.columns:
    means[ch] = []
    ranges[ch] = []
    theils[ch] = []
    simpsons[c] = []

In [16]:
gdf['lcdMes'] = gdf.apply(
            lambda row: row.lcdMes if row.lcdMes >= 0 else 0,
            axis=1,
        )  # normally does not happen, but to be sure

In [17]:
gdf = gdf.round(6)
gdf = gdf.set_index('uID')

In [ ]:
for index in tqdm(range(len(gdf)), total=gdf.shape[0]):
    neighbours = [index]
    neighbours += spatial_weights.neighbors[index]
    
    subset = gdf.iloc[neighbours]
    for ch in gdf.columns:
        values_list = subset[ch] 
        idec = limit_range(values_list, rng=(10, 90))
        iquar = limit_range(values_list, rng=(25, 75))
        
        means[ch].append(np.mean(iquar))
        ranges[ch].append(sp.stats.iqr(values_list, rng=(25, 75)))
        theils[ch].append(theil(idec))
        
        sample_bins = classifiers.UserDefined(values_list, list(bins[ch]))
        counts = dict(zip(bins[ch], sample_bins.counts))
        simpsons[ch].append(_simpson_di(counts))

  0%|          | 0/506435 [00:00<?, ?it/s]/opt/conda/lib/python3.8/site-packages/mapclassify/classifiers.py:888: RuntimeWarning: invalid value encountered in double_scalars
  gadf = 1 - self.adcm / adam
 10%|█         | 51102/506435 [1:03:52<9:35:27, 13.19it/s] 

In [23]:
gdf.columns

Index(['uID', 'stcOri', 'sdcLAL', 'sdcAre', 'sscCCo', 'sscERI', 'sicCAR',
       'mtcWNe', 'mdcAre', 'ltcWRB', 'stcSAl', 'sdbAre', 'sdbPer', 'ssbCCo',
       'ssbCor', 'ssbSqu', 'ssbERI', 'ssbElo', 'ssbCCM', 'ssbCCD', 'stbOri',
       'stbCeA', 'mtbAli', 'mtbNDi', 'ltbIBD', 'ltcBuA', 'stbSAl', 'ldkAre',
       'ldkPer', 'lskCCo', 'lskERI', 'lskCWA', 'ltkOri', 'ltkWNB', 'likWBB',
       'sdsLen', 'sdsSPW', 'sdsSPO', 'sdsSWD', 'sssLin', 'sdsAre', 'sisBpM',
       'misRea', 'mdsAre', 'ldsMSL', 'ldsRea', 'mtdDeg', 'lcdMes', 'linP3W',
       'linP4W', 'linPDE', 'lcnClo', 'ldsCDL', 'xcnSCl', 'mtdMDi', 'lddNDe',
       'linWID', 'lddRea', 'lddARe', 'sddAre', 'midRea', 'midAre', 'sscERI_r',
       'ssbERI_r', 'ltcBuA_r', 'sssLin_r', 'uID_meanIQ3', 'uID_rangeIQ3',
       'uID_theilID3'],
      dtype='object')

In [25]:
means.keys()

TypeError: 'dict_keys' object is not subscriptable

In [28]:
for ch in means.keys():
    print(ch)
    gdf[ch + '_meanIQ3'] = means[ch]
    gdf[ch + '_rangeIQ3'] = ranges[ch]
    gdf[ch + '_theilID3'] = theils[ch]
    gdf[c + '_simpson'] = simpsons[ch]

uID


KeyError: 'uID'

In [ ]:
pat = [x for x in gdf.columns if '_' in x]
gdf2 = gdf[[x for x in pat if 'uID' not in x]].reset_index()

In [ ]:
gdf2.to_parquet('../../nairobi/contextual_10.parquet')

In [26]:
primary

,uID,stcOri,sdcLAL,sdcAre,sscCCo,sscERI,sicCAR,mtcWNe,mdcAre,ltcWRB,...,ldsCDL,xcnSCl,mtdMDi,lddNDe,linWID,lddRea,lddARe,sddAre,midRea,midAre
0,0,1.611360,119.156839,4151.756119,0.372309,1.027104,0.003613,0.017359,27230.630199,0.000014,...,103.723138,0.00,251.409326,0.003414,0.010242,143.0,4.424674e+05,110958.301197,35.0,167414.385903
1,1,0.269122,111.328604,4414.560298,0.453507,1.022801,0.002605,0.014119,15611.059576,0.000022,...,172.632512,0.00,33.145996,0.009738,0.033476,109.0,9.360662e+04,12864.563865,27.0,32307.842657
2,2,4.043140,121.628792,7338.416372,0.631596,1.037698,0.002964,0.012019,25150.210765,0.000015,...,152.302990,0.00,152.302990,0.011908,0.046441,60.0,3.865676e+04,14841.570413,29.0,21592.408884
3,3,0.559098,126.719262,4560.875763,0.361637,1.014886,0.003124,0.012894,18411.086095,0.000043,...,222.359525,0.00,164.190257,0.005921,0.019645,148.0,3.350288e+05,37210.425533,42.0,106854.000623
4,4,0.518096,114.291541,5340.439428,0.520546,1.053450,0.002387,0.017010,19801.649928,0.000025,...,317.637414,0.00,149.167849,0.097657,0.313896,398.0,1.233273e+06,49389.581385,41.0,91432.394862
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
506430,507527,2.487274,210.237018,20392.478599,0.587438,1.005770,0.002758,0.006981,83218.427805,0.000005,...,2286.739202,0.00,558.868006,0.002662,0.010650,141.0,1.576907e+06,426569.673852,65.0,947313.509028
506431,507528,2.302916,162.323252,9622.811292,0.464998,1.008737,0.033761,0.014532,32473.689409,0.000028,...,0.000000,0.00,219.945297,0.008168,0.029788,42.0,1.804964e+05,34131.291457,15.0,76686.457491
506432,507529,1.186238,280.844722,28977.337582,0.467774,0.860495,0.376018,0.015148,95345.201325,0.000010,...,303.602588,0.25,55.806120,0.013120,0.041746,43.0,1.193281e+05,28977.337582,2.0,30844.699639
506433,507530,3.001379,188.010383,10016.219927,0.360787,0.971733,0.038887,0.008743,30066.804465,0.000038,...,0.000000,0.00,257.864740,0.005189,0.014432,35.0,3.286643e+05,72776.767711,14.0,86233.695861
